In [1]:
#import the GCMP biom table and split it into 3 feature tables, one for
#each compartment

In [8]:
from qiime2 import Artifact
from qiime2.plugins.feature_table.methods import filter_samples
from qiime2.metadata import Metadata

In [3]:
#import the feature table
#import the sequences (need upper?)
#filter the sequences by the feature table
#filter the sequences and feature table by metadata (needed?)

In [4]:
working_dir = '/home/dylan/Documents/june_reset'

In [5]:
#import the biom table as a feature table
biom_path = working_dir + '/input/all.biom'
GCMP_ft = Artifact.import_data('FeatureTable[Frequency]', biom_path,
                               'BIOMV210Format')
#import_data(type, view, view_type=None)

In [10]:
#import the metadata
metadata_path = working_dir + '/input/GCMP_EMP_map_r28_no_empty_samples.txt'
metadata = Metadata.load(metadata_path)

In [13]:
#split into mucus, tissue, and skeleton compartments
compartments = ['M', 'T', 'S']
for compartment in compartments:
    where = "tissue_compartment='" + compartment + "'"
    GCMP_filtered, = filter_samples(GCMP_ft, metadata = metadata,
                                   where = where)
    save_path = working_dir + '/input/' + compartment + "_ft.qza"
    GCMP_filtered.save(save_path)

In [ ]:
#import the sequences
#mix of upper and lowercase; this version of qiime is not a fan
seqs_path = working_dir + '/input/all.seqs.fa'
upper_seqs_path = working_dir + '/input/GCMP.fasta'
with open(seqs_path) as infile:
    with open(upper_seqs_path, "a") as outfile:
        for line in infile:
            #careful to preserve the headers so they match the feature table
            if not line.startswith(">"):
                line = line.upper()
            outfile.write(line)

In [14]:
#now we can import, and save the artifact
GCMP_seqs = Artifact.import_data('FeatureData[Sequence]', upper_seqs_path)
GCMP_seqs.save(working_dir + '/input/GCMP_seqs.qza')

'/home/dylan/Documents/june_reset/input/GCMP_seqs.qza'